In [1]:
import pandas as pd
import numpy as np

import jieba

import torch

from datasets import Dataset, load_metric

from sklearn.model_selection import train_test_split
from transformers import AutoTokenizer, AutoModelForSeq2SeqLM, DataCollatorForSeq2Seq, Seq2SeqTrainingArguments, Seq2SeqTrainer

/Users/keejuan/Documents/ML/CNENTranslation/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


Preprocess data.

In [2]:
def load_data(path):
    with open(path, 'r', encoding='utf-8') as f:
        # replace '\n' with ' '
        data = pd.Series(f.readlines())
        data = data.str.replace('\n', ' ')
    return data

In [3]:
data_ZH = load_data('chinese.zh')
data_EN = load_data('english.en')

In [4]:
data_ZH = data_ZH[:10000]
data_EN = data_EN[:10000]

In [5]:
def cut_ZH(data):
    data_cut = data.apply(lambda x: ' '.join(jieba.cut(x)))
    return data_cut

data_ZH = cut_ZH(data_ZH)
data_ZH

Building prefix dict from the default dictionary ...
Loading model from cache /var/folders/s8/pbv_qcy127g2gt_nwhfbrnxr0000gn/T/jieba.cache
Loading model cost 0.339 seconds.
Prefix dict has been built successfully.


0                                    1929 年 还是 1989 年 ?  
1       巴黎 - 随着 经济危机 不断 加深 和 蔓延 ， 整个 世界 一直 在 寻找 历史 上 的...
2       一 开始 ， 很多 人 把 这次 危机 比作 1982 年 或 1973 年 所 发生 的 ...
3       如今 人们 的 心情 却是 沉重 多 了 ， 许多 人 开始 把 这次 危机 与 1929 ...
4       目前 的 趋势 是 ， 要么 是 过度 的 克制 （ 欧洲 ） ， 要么 是 努力 的 扩展...
                              ...                        
9995    根据 像 约翰 · 麦凯恩 这样 的 人 的 观点 ， 最 根本 的 原因 是 华尔街 的 ...
9996    虽然 我 不 否认 这样 的 基础 动力 的 存在 ， 但是 ， 我 还是 坚持 认为 这场...
9997                              在 美国 ， 有 两个 关键 的 决定 。  
9998    第一个 是 在 20 世纪 70 年代 解除 了 对 支付 给 股票 经纪人 的 佣金 的 ...
9999    第二个 是 在 20 世纪 90 年代 废止 了 《 格拉斯 - 斯蒂格 尔法 》 对 商业...
Length: 10000, dtype: object

In [6]:
df_ZH = data_ZH.to_frame()
df_EN = data_EN.to_frame()

Constants and helper functions.

In [9]:
ENGLISH = 'en'
CHINESE = 'zh'
PREFIX = ""
MAX_INPUT_LENGTH = 128
MAX_TARGET_LENGTH = 128
INPUT_IDS = "input_ids"
LABELS = "labels"
BATCH_SIZE = 16

In [10]:
def postprocess_text(preds: list, labels: list) -> tuple:
    """Performs post processing on the prediction text and labels"""

    preds = [pred.strip() for pred in preds]
    labels = [[label.strip()] for label in labels]

    return preds, labels

In [11]:
def prep_data_for_model_fine_tuning(source_lang: list, target_lang: list) -> list:
    """Takes the input data lists and converts into translation list of dicts"""

    data_dict = dict()
    data_dict['translation'] = []

    for sr_text, tr_text in zip(source_lang, target_lang):
        temp_dict = dict()
        temp_dict[CHINESE] = sr_text
        temp_dict[ENGLISH] = tr_text

        data_dict['translation'].append(temp_dict)

    return data_dict


In [12]:
def generate_model_ready_dataset(dataset: list, source: str, target: str,
                                 model_checkpoint: str,
                                 tokenizer: AutoTokenizer):
    """Makes the data training ready for the model"""

    preped_data = []

    for row in dataset:
        inputs = (PREFIX + row[source][0])
        targets = (row[target][0])

        model_inputs = tokenizer(inputs, max_length=MAX_INPUT_LENGTH,
                                 truncation=True, padding=True)

        model_inputs['translation'] = row

        # setup the tokenizer for targets
        with tokenizer.as_target_tokenizer():
            labels = tokenizer(targets, max_length=MAX_INPUT_LENGTH,
                                 truncation=True, padding=True)
            model_inputs[LABELS] = labels[INPUT_IDS]

        preped_data.append(model_inputs)

    return preped_data

In [18]:
def compute_metrics(eval_preds: tuple) -> dict:
    """computes bleu score and other performance metrics """

    metric = load_metric("sacrebleu")
    tokenizer = AutoTokenizer.from_pretrained("Helsinki-NLP/opus-mt-zh-en")

    preds, labels = eval_preds

    if isinstance(preds, tuple):
        preds = preds[0]

    decoded_preds = tokenizer.batch_decode(preds, skip_special_tokens=True)

    # Replace -100 in the labels as we can't decode them.
    labels = np.where(labels != -100, labels, tokenizer.pad_token_id)
    decoded_labels = tokenizer.batch_decode(labels, skip_special_tokens=True)

    # Some simple post-processing
    decoded_preds, decoded_labels = postprocess_text(decoded_preds, decoded_labels)

    result = metric.compute(predictions=decoded_preds, references=decoded_labels)
    result = {'bleu': result['score']}

    prediction_lens = [np.count_nonzero(pred != tokenizer.pad_token_id) for pred in preds]

    result['gen_len'] = np.mean(prediction_lens)
    result = {k: round(v, 4) for k, v in result.items()}

    return result

Perform train test split.

In [7]:
X = df_ZH
y = df_EN

In [8]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.10, shuffle = True, random_state = 100)
X_train, X_val, y_train, y_val = train_test_split(X_train, y_train, test_size = 0.20, shuffle = True, random_state = 100)

Load tokenizer. Using Hensinki-NLP Chinese to English.

In [13]:
tokenizer = AutoTokenizer.from_pretrained("Helsinki-NLP/opus-mt-zh-en")

Process dataset to make it suitable for model, including tokenisation and converting it into a Dataset object.

In [14]:
# Prepare training, validation, and test data
training_data = prep_data_for_model_fine_tuning(X_train.values, y_train.values)
validating_data = prep_data_for_model_fine_tuning(X_val.values, y_val.values)
testing_data = prep_data_for_model_fine_tuning(X_test.values, y_test.values)

In [15]:
# Generate model-ready inputs for training, validation, and test
train_data = generate_model_ready_dataset(dataset=training_data['translation'],
                                          tokenizer=tokenizer,
                                          source=CHINESE,
                                          target=ENGLISH,
                                          model_checkpoint='Helsinki-NLP/opus-mt-zh-en')
validation_data = generate_model_ready_dataset(dataset=validating_data['translation'],
                                               tokenizer=tokenizer,
                                               source=CHINESE,
                                               target=ENGLISH,
                                               model_checkpoint='Helsinki-NLP/opus-mt-zh-en')
test_data = generate_model_ready_dataset(dataset=testing_data['translation'],
                                         tokenizer=tokenizer,
                                         source=CHINESE,
                                         target=ENGLISH,
                                         model_checkpoint='Helsinki-NLP/opus-mt-zh-en')

/Users/keejuan/Documents/ML/CNENTranslation/lib/python3.12/site-packages/transformers/tokenization_utils_base.py:3935: UserWarning: `as_target_tokenizer` is deprecated and will be removed in v5 of Transformers. You can tokenize your labels by using the argument `text_target` of the regular `__call__` method (either in the same call as your input texts if you use the same keyword arguments, or in a separate call.
  warnings.warn(


In [16]:
# Convert to DataFrame
train_df = pd.DataFrame.from_records(train_data)
validation_df = pd.DataFrame.from_records(validation_data)
test_df = pd.DataFrame.from_records(test_data)

# Convert DataFrames to Dataset objects
train_dataset = Dataset.from_pandas(train_df)
validation_dataset = Dataset.from_pandas(validation_df)
test_dataset = Dataset.from_pandas(test_df)

Prepare model.

In [17]:
# Load the pre-trained model and define training arguments
model = AutoModelForSeq2SeqLM.from_pretrained("Helsinki-NLP/opus-mt-zh-en")

model_args = Seq2SeqTrainingArguments(
    "translation-pt-en-t5-finetuned-zh-to-en",
    evaluation_strategy='epoch',
    learning_rate=2e-4,
    per_device_train_batch_size=BATCH_SIZE,
    per_device_eval_batch_size=BATCH_SIZE,
    weight_decay=0.02,
    save_total_limit=3,
    num_train_epochs=7,
    predict_with_generate=True
)

# Create a data collator for sequence-to-sequence tasks
data_collator = DataCollatorForSeq2Seq(tokenizer, model=model)

In [19]:
data_collator = DataCollatorForSeq2Seq(tokenizer, model=model)

Train model.

In [20]:
trainer = Seq2SeqTrainer(
    model,
    model_args,
    train_dataset=train_dataset,
    eval_dataset=validation_dataset,
    data_collator=data_collator,
    tokenizer=tokenizer,
    compute_metrics=compute_metrics
)

/Users/keejuan/Documents/ML/CNENTranslation/lib/python3.12/site-packages/accelerate/accelerator.py:432: FutureWarning: Passing the following arguments to `Accelerator` is deprecated and will be removed in version 1.0 of Accelerate: dict_keys(['dispatch_batches', 'split_batches', 'even_batches', 'use_seedable_sampler']). Please pass an `accelerate.DataLoaderConfiguration` instead: 
dataloader_config = DataLoaderConfiguration(dispatch_batches=None, split_batches=False, even_batches=True, use_seedable_sampler=True)
  warnings.warn(


In [21]:
trainer.train()

Epoch,Training Loss,Validation Loss,Blue,Gen Len
1,No log,1.980045,21.802200,28.609400
2,1.974800,2.019087,21.907500,29.283300
3,1.218500,2.128166,22.105300,28.985000
4,0.743300,2.228374,22.449400,28.942200
5,0.440600,2.326017,22.683000,28.571700
6,0.248400,2.400083,22.699100,28.856100
7,0.139100,2.444219,22.971000,29.079400


/var/folders/s8/pbv_qcy127g2gt_nwhfbrnxr0000gn/T/ipykernel_93670/1112429398.py:4: FutureWarning: load_metric is deprecated and will be removed in the next major version of datasets. Use 'evaluate.load' instead, from the new library 🤗 Evaluate: https://huggingface.co/docs/evaluate
  metric = load_metric("sacrebleu")
/Users/keejuan/Documents/ML/CNENTranslation/lib/python3.12/site-packages/datasets/load.py:756: FutureWarning: The repository for sacrebleu contains custom code which must be executed to correctly load the metric. You can inspect the repository content at https://raw.githubusercontent.com/huggingface/datasets/2.18.0/metrics/sacrebleu/sacrebleu.py
You can avoid this message in future by passing the argument `trust_remote_code=True`.
Passing `trust_remote_code=True` will be mandatory to load this metric from the next major release of `datasets`.
  warnings.warn(
Some non-default generation parameters are set in the model config. These should go into a GenerationConfig file (htt

TrainOutput(global_step=3150, training_loss=0.7614415686471122, metrics={'train_runtime': 10415.4698, 'train_samples_per_second': 4.839, 'train_steps_per_second': 0.302, 'total_flos': 1238519135600640.0, 'train_loss': 0.7614415686471122, 'epoch': 7.0})

In [39]:
# Save the fine-tuned model
trainer.save_model("FineTunedTranslationModel")

Some non-default generation parameters are set in the model config. These should go into a GenerationConfig file (https://huggingface.co/docs/transformers/generation_strategies#save-a-custom-decoding-strategy-with-your-model) instead. This warning will be raised to an exception in v4.41.
Non-default generation parameters: {'max_length': 512, 'num_beams': 6, 'bad_words_ids': [[65000]], 'forced_eos_token_id': 0}


Test model.

In [46]:
model = model.to('mps')

In [47]:
test_results = trainer.predict(test_dataset)

/Users/keejuan/Documents/ML/CNENTranslation/lib/python3.12/site-packages/datasets/load.py:756: FutureWarning: The repository for sacrebleu contains custom code which must be executed to correctly load the metric. You can inspect the repository content at https://raw.githubusercontent.com/huggingface/datasets/2.18.0/metrics/sacrebleu/sacrebleu.py
You can avoid this message in future by passing the argument `trust_remote_code=True`.
Passing `trust_remote_code=True` will be mandatory to load this metric from the next major release of `datasets`.
  warnings.warn(


In [48]:
print("Test Bleu Score: ", test_results.metrics["test_blue"])

Test Bleu Score:  22.7497


Generate predicted sentences of test dataset for human evaluation.

In [49]:
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
model.to(device)

MarianMTModel(
  (model): MarianModel(
    (shared): Embedding(65001, 512, padding_idx=65000)
    (encoder): MarianEncoder(
      (embed_tokens): Embedding(65001, 512, padding_idx=65000)
      (embed_positions): MarianSinusoidalPositionalEmbedding(512, 512)
      (layers): ModuleList(
        (0-5): 6 x MarianEncoderLayer(
          (self_attn): MarianAttention(
            (k_proj): Linear(in_features=512, out_features=512, bias=True)
            (v_proj): Linear(in_features=512, out_features=512, bias=True)
            (q_proj): Linear(in_features=512, out_features=512, bias=True)
            (out_proj): Linear(in_features=512, out_features=512, bias=True)
          )
          (self_attn_layer_norm): LayerNorm((512,), eps=1e-05, elementwise_affine=True)
          (activation_fn): SiLU()
          (fc1): Linear(in_features=512, out_features=2048, bias=True)
          (fc2): Linear(in_features=2048, out_features=512, bias=True)
          (final_layer_norm): LayerNorm((512,), eps=1e-05

In [50]:
from tqdm import tqdm

predictions = []
test_input = test_dataset['translation']

for input_text in tqdm(test_input):
    source_sentence = input_text[CHINESE]
    encoded_source = tokenizer(source_sentence,
                               return_tensors='pt',
                               padding=True,
                               truncation=True)
    encoded_source.to(device)  # Move input tensor to the same device as the model

    translated = model.generate(**encoded_source)

    predictions.append([tokenizer.decode(t, skip_special_tokens=True) for t in translated][0])

# Move the model back to CPU if needed
model.to("cpu")

100%|█████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 1000/1000 [18:39<00:00,  1.12s/it]


MarianMTModel(
  (model): MarianModel(
    (shared): Embedding(65001, 512, padding_idx=65000)
    (encoder): MarianEncoder(
      (embed_tokens): Embedding(65001, 512, padding_idx=65000)
      (embed_positions): MarianSinusoidalPositionalEmbedding(512, 512)
      (layers): ModuleList(
        (0-5): 6 x MarianEncoderLayer(
          (self_attn): MarianAttention(
            (k_proj): Linear(in_features=512, out_features=512, bias=True)
            (v_proj): Linear(in_features=512, out_features=512, bias=True)
            (q_proj): Linear(in_features=512, out_features=512, bias=True)
            (out_proj): Linear(in_features=512, out_features=512, bias=True)
          )
          (self_attn_layer_norm): LayerNorm((512,), eps=1e-05, elementwise_affine=True)
          (activation_fn): SiLU()
          (fc1): Linear(in_features=512, out_features=2048, bias=True)
          (fc2): Linear(in_features=2048, out_features=512, bias=True)
          (final_layer_norm): LayerNorm((512,), eps=1e-05

In [51]:
y_true_en = []
y_true_pt = []

for input_text in tqdm(test_input):
    y_true_pt.append(input_text[CHINESE])
    y_true_en.append(input_text[ENGLISH])

100%|█████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 1000/1000 [00:00<00:00, 143493.12it/s]


In [52]:
output_df = pd.DataFrame({"y_true_port": y_true_pt, "y_true_eng": y_true_en, "predicted_text": predictions})
output_df

,y_true_port,y_true_eng,predicted_text
0,[这些 特殊 利益 包括 华尔街 、 大 石油 公司 、 大 人寿保险 商 以及 军火商 。...,"[These special interests included Wall Street,...","Such special interests include Wall Street, la..."
1,[经过 多年 的 争论 和 布什 政府 反对 采取任何 行动 的 态度 影响 ， 美国 终于...,"[After years of debate in the United States, a...",After years of wrangling and the Bush administ...
2,[有 数百万 东南亚 人 冒险 到 中东 工作 ， 但是 还有 数百万 人 跨境 到 该 地...,[Millions of people in Southeast Asia venture ...,Many millions of Southeast Asians risk working...
3,[值得一提的是 三位 前 拉美 总统 ， 墨西哥 排名 第二 的 富翁 里卡多 · 萨利纳斯...,"[Three former Latin American presidents, Mexic...",Mention should be taken of three former Latin ...
4,[肥料 和 杀虫剂 的 价格 都 比 我 想象 的 要 贵 。 ],[Fertilizer and spraying were both more expens...,The price of fertilizer and pesticides is more...
...,...,...,...
995,[解决问题 需要 呼吁 增加 税收 ， 而 乔治 · W · 布什 1992 年 得到 的 ...,[To do so would require calling for higher tax...,Solving the problem requires an increase in ta...
996,[美国 的 战略 真空 ],[America’s Strategy Vacuum ],America’s Strategy Gap
997,[根据 对 7000 家 公司 所 做 的 月度 调查 ， Ifo 的 德国 经济 景气 指...,"[The Ifo climate indicator for Germany, based ...","According to a monthly survey of 700,000 compa..."
998,[其一 是 移民 改革 ， 这个 问题 在 美国 与 在 欧洲 一样 存在 争议 。 ],"[One is immigration reform, which is as contro...","One is immigration reform, a matter that is as..."


In [53]:
output_df.to_csv('translation.csv', index=False, encoding='utf_8_sig')